# 박스오피스 데이터 크롤링

http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=430156241533f1d058c603178cc3ca0e&targetDt=20181030

In [1]:
def daily_boxoffice(date=20181030):
    
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList\
.json?key=430156241533f1d058c603178cc3ca0e&targetDt={}'.format(date)
    response = requests.get(url)
    results = response.json()
    
    daily = pd.DataFrame(results['boxOfficeResult']['dailyBoxOfficeList'])
    daily = daily.drop(axis=1,columns={'rnum','salesInten','movieCd','audiInten','audiChange','rankInten',
                                   'rankOldAndNew','salesShare','openDt','salesChange'})
    daily = daily.rename(columns={"audiAcc":"누적관객수","audiCnt":"관객수","movieNm":"영화제목","rank":"순위",
                              "salesAcc":"누적수익","salesAmt":'수익',"scrnCnt":"상영스크린수","showCnt":"상영횟수"})
    movie = daily["영화제목"]
    rank = daily["순위"]
    daily.drop(["영화제목","순위"], axis=1, inplace=True)
    daily.insert(loc=0, column='영화제목', value=movie)
    daily.insert(loc=0, column='순위', value=rank)
    
    return daily

In [2]:
boxoffice_150101 = daily_boxoffice(20150101)
boxoffice_150101

,순위,영화제목,누적관객수,관객수,누적수익,수익,상영스크린수,상영횟수
0,1,국제시장,6097431,751754,47219550096,6129982500,941,4650
1,2,테이큰 3,467280,321653,3658460100,2644551100,614,2947
2,3,마다가스카의 펭귄,361669,212779,2736013000,1687516200,594,1846
3,4,"님아, 그 강을 건너지 마오",4025319,178941,31324845079,1479015600,473,1979
4,5,기술자들,1938454,160371,14952363848,1299883700,464,1886
5,6,호빗: 다섯 군대 전투,2579875,69988,22038162144,609678800,346,825
6,7,개를 훔치는 완벽한 방법,80633,37741,608002300,299346400,196,650
7,8,눈의 여왕 2: 트롤의 마법거울,529298,30978,3856612400,233190700,246,415
8,9,상의원,705071,28439,5524209364,230484800,258,491
9,10,인터스텔라,10125883,20636,80863567400,192143100,97,175


In [3]:
boxoffice_170101 = daily_boxoffice(20170101)
boxoffice_170101

,순위,영화제목,누적관객수,관객수,누적수익,수익,상영스크린수,상영횟수
0,1,마스터,5441619,506299,44184434165,4399793100,1245,5721
1,2,로그 원: 스타워즈 스토리,769938,173451,6739961794,1612424700,810,3335
2,3,씽,1178327,115152,9147833289,943975500,721,1755
3,4,판도라,4401830,87966,34692290064,736891900,576,1476
4,5,라라랜드,2472637,87529,20767028832,770889600,590,1578
5,6,너의 이름은.,74181,37418,640998500,325811500,275,342
6,7,루돌프와 많이있어,119420,29910,926773600,243890000,372,581
7,8,포켓몬 더 무비 XY&Z 「볼케니온 : 기계왕국의 비밀」,238452,19405,1821568800,154931400,300,422
8,9,"당신, 거기 있어줄래요",1143445,18630,9165307248,166222000,261,572
9,10,극장판 도라에몽: 신 진구의 버스 오브 재팬,62699,3552,482548500,28628400,97,107


# 네이버 영화 평점 크롤링(selenium)

In [4]:
movies = ['창궐', '완벽한 타인', '암수살인', '스타 이즈 본', '바울', '미쓰백']

In [5]:
def movie_scores(movies):
    
    response = requests.get("https://movie.naver.com/")
    dom = bs(response.content, 'html.parser')
    
    ntz_scores = []
    spc_scores = []
    
    for movie in movies:
        
        dom.select_one('.ipt_srch > .ipt_tx_srch').send_keys(movie)
        dom.select_one('.btn_srch').click()
        dom.select_one('.search_list_1 > li > dl > dt > a').click()

        ntz_score = dom.select('#actualPointPersentBasic > .star_score > em')
        ntz_score = ''.join([el.text for el in ntz_score])
        ntz_scores.append(ntz_score)

        spc_score = dom.select('.score > .spc_score_area > .spc > .star_score > em')
        spc_score = ''.join([el.text for el in spc_score])
        spc_scores.append(spc_score)
    
    score_df = pd.DataFrame(ntz_scores, spc_scores)
 
    return score_df
    

In [6]:
def naver_movie_score(movies, headless=True):
    
    # 헤드리스 설정
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('headless')
        
    driver = webdriver.Chrome(options=options)
    driver.get('https://movie.naver.com/')

    ntz_scores = [] # 관람객 평점
    spc_scores = [] # 기자,평론가 평점
    
    score_df = pd.DataFrame(columns=['관람객 평점', '기자.평론가 평점']) # 데이터프레임 생성
    
    for movie in movies:

        # 영화 제목 검색
        driver.find_element_by_css_selector('.ipt_srch > .ipt_tx_srch').send_keys(movie) 
        driver.find_element_by_css_selector('.btn_srch').click()
        driver.find_element_by_css_selector('.search_list_1 > li > dl > dt > a').click()

        # 관람객 평점 크롤링
        ntz_score = driver.find_elements_by_css_selector('#actualPointPersentBasic > .star_score > em')
        ntz_score = ''.join([el.text for el in ntz_score])
        ntz_scores.append(ntz_score)

        # 기자,평론가 평점 크롤링
        spc_score = driver.find_elements_by_css_selector('.score > .spc_score_area > .spc > .star_score > em')
        spc_score = ''.join([el.text for el in spc_score])
        spc_scores.append(spc_score)
        
    driver.quit()    

    score_df['관람객 평점'] = ntz_scores
    score_df['기자.평론가 평점'] = spc_scores
    

    return score_df

In [7]:
movies = ['창궐', '완벽한 타인', '암수살인', '스타 이즈 본', '바울', '미쓰백']

In [8]:
naver_movie_score(movies)

,관람객 평점,기자.평론가 평점
0,6.51,5.20
1,9.08,6.13
2,8.58,8.14
3,9.15,7.43
4,9.48,5.00
5,9.25,6.11
